In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6862,2021-01-15T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1403,210,1613,43432,...,2916,NaN,NaN,117336,1367989,894456.0,NaN,18.0,NaN,NaN
6863,2021-01-15T17:00:00,ITA,9,Toscana,43.769231,11.255889,715,129,844,7754,...,3940,NaN,NaN,127010,2012434,1106215.0,NaN,7.0,NaN,NaN
6864,2021-01-15T17:00:00,ITA,10,Umbria,43.106758,12.388247,278,50,328,4161,...,682,NaN,NaN,31909,543318,265261.0,Si fa presente come uno dei ricoveri in terapi...,3.0,NaN,NaN
6865,2021-01-15T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,40,3,43,373,...,392,NaN,NaN,7601,66212,40288.0,NaN,2.0,NaN,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6862,2021-01-15T17:00:00,19,Sicilia,1403,210,1613,43432,45045,180,1945,69375,2916,NaN,NaN,117336,1367989,894456.0,18.0,NaN,NaN
6863,2021-01-15T17:00:00,9,Toscana,715,129,844,7754,8598,-87,446,114472,3940,NaN,NaN,127010,2012434,1106215.0,7.0,NaN,NaN
6864,2021-01-15T17:00:00,10,Umbria,278,50,328,4161,4489,0,232,26738,682,NaN,NaN,31909,543318,265261.0,3.0,NaN,NaN
6865,2021-01-15T17:00:00,2,Valle d'Aosta,40,3,43,373,416,4,20,6793,392,NaN,NaN,7601,66212,40288.0,2.0,NaN,NaN


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-01-15', '2021-01-14')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2021-01-15T17:00:00,1,Piemonte,2522,163,2685,12121,14806,83,871,188084,8319,NaN,NaN,211209,1785825,1025418.0,8.0,NaN,NaN
1,2021-01-15T17:00:00,2,Valle d'Aosta,40,3,43,373,416,4,20,6793,392,NaN,NaN,7601,66212,40288.0,2.0,NaN,NaN
2,2021-01-15T17:00:00,3,Lombardia,3601,466,4067,53087,57154,1054,2205,427181,26094,NaN,NaN,510429,5171810,2713639.0,19.0,NaN,NaN
3,2021-01-15T17:00:00,5,Veneto,2386,327,2713,72640,75353,-4472,1079,211662,7859,NaN,NaN,294874,3584775,1278230.0,32.0,NaN,NaN
4,2021-01-15T17:00:00,6,Friuli Venezia Giulia,688,68,756,12123,12879,16,919,44597,2035,NaN,NaN,59511,1022612,365548.0,6.0,NaN,NaN
5,2021-01-15T17:00:00,7,Liguria,668,59,727,4307,5034,-40,254,57096,3084,NaN,NaN,65214,764447,340315.0,0.0,I 2243 test antigenici rapidi sono riferiti al...,NaN
6,2021-01-15T17:00:00,8,Emilia-Romagna,2511,231,2742,52768,55510,-438,1794,134711,8657,NaN,NaN,198878,2774816,1289999.0,10.0,NaN,NaN
7,2021-01-15T17:00:00,9,Toscana,715,129,844,7754,8598,-87,446,114472,3940,NaN,NaN,127010,2012434,1106215.0,7.0,NaN,NaN
8,2021-01-15T17:00:00,10,Umbria,278,50,328,4161,4489,0,232,26738,682,NaN,NaN,31909,543318,265261.0,3.0,NaN,NaN
9,2021-01-15T17:00:00,11,Marche,587,82,669,11020,11689,-504,629,35810,1763,NaN,NaN,49262,606048,362659.0,9.0,NaN,NaN


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2021-01-14T17:00:00,1,Piemonte,2575,172,2747,11976,14723,-37,857,187316,8299,NaN,NaN,210338,1767779,1020572.0,14.0,NaN,NaN
1,2021-01-14T17:00:00,2,Valle d'Aosta,43,2,45,367,412,-8,16,6777,392,NaN,NaN,7581,65958,40219.0,0.0,NaN,NaN
2,2021-01-14T17:00:00,3,Lombardia,3614,468,4082,52018,56100,1137,2587,426098,26026,NaN,NaN,508224,5137863,2704477.0,26.0,NaN,NaN
3,2021-01-14T17:00:00,5,Veneto,2447,328,2775,77050,79825,-2145,2076,206185,7785,NaN,NaN,293795,3526933,1274187.0,32.0,NaN,NaN
4,2021-01-14T17:00:00,6,Friuli Venezia Giulia,668,68,736,12127,12863,65,805,43723,2006,NaN,NaN,58592,1011978,361869.0,5.0,NaN,NaN
5,2021-01-14T17:00:00,7,Liguria,678,65,743,4331,5074,72,289,56812,3074,NaN,NaN,64960,758219,338106.0,5.0,I 191219 test antigenici rapidi sono riferiti ...,NaN
6,2021-01-14T17:00:00,8,Emilia-Romagna,2610,233,2843,53105,55948,-2032,1515,132548,8590,NaN,NaN,197086,2752436,1285504.0,13.0,NaN,NaN
7,2021-01-14T17:00:00,9,Toscana,723,132,855,7830,8685,-141,424,113949,3930,NaN,NaN,126564,1998217,1103382.0,2.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2021-01-14T17:00:00,10,Umbria,277,51,328,4161,4489,19,210,26510,678,NaN,NaN,31677,539805,264287.0,3.0,NaN,NaN
9,2021-01-14T17:00:00,11,Marche,583,76,659,11534,12193,-965,506,34687,1753,NaN,NaN,48633,597883,358740.0,8.0,NaN,NaN


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2021-01-15T17:00:00,1,Piemonte,2522,163,2685,12121,14806,83,871,188084,8319,NaN,NaN,211209,1785825,1025418.0,8.0,NaN,NaN
1,2021-01-15T17:00:00,2,Valle d'Aosta,40,3,43,373,416,4,20,6793,392,NaN,NaN,7601,66212,40288.0,2.0,NaN,NaN
2,2021-01-15T17:00:00,3,Lombardia,3601,466,4067,53087,57154,1054,2205,427181,26094,NaN,NaN,510429,5171810,2713639.0,19.0,NaN,NaN
3,2021-01-15T17:00:00,4,Trentino-Alto Adige,513,65,578,13537,14115,432,891,41499,1853,0.0,0.0,57467,873851,326301.0,6.0,0,NaN
4,2021-01-15T17:00:00,5,Veneto,2386,327,2713,72640,75353,-4472,1079,211662,7859,NaN,NaN,294874,3584775,1278230.0,32.0,NaN,NaN
5,2021-01-15T17:00:00,6,Friuli Venezia Giulia,688,68,756,12123,12879,16,919,44597,2035,NaN,NaN,59511,1022612,365548.0,6.0,NaN,NaN
6,2021-01-15T17:00:00,7,Liguria,668,59,727,4307,5034,-40,254,57096,3084,NaN,NaN,65214,764447,340315.0,0.0,I 2243 test antigenici rapidi sono riferiti al...,NaN
7,2021-01-15T17:00:00,8,Emilia-Romagna,2511,231,2742,52768,55510,-438,1794,134711,8657,NaN,NaN,198878,2774816,1289999.0,10.0,NaN,NaN
8,2021-01-15T17:00:00,9,Toscana,715,129,844,7754,8598,-87,446,114472,3940,NaN,NaN,127010,2012434,1106215.0,7.0,NaN,NaN
9,2021-01-15T17:00:00,10,Umbria,278,50,328,4161,4489,0,232,26738,682,NaN,NaN,31909,543318,265261.0,3.0,NaN,NaN


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,0,2021-01-14T17:00:00,1,Piemonte,2575,172,2747,11976,14723,-37,...,187316,8299,NaN,NaN,210338,1767779,1020572.0,14.0,NaN,NaN
1,1,2021-01-14T17:00:00,2,Valle d'Aosta,43,2,45,367,412,-8,...,6777,392,NaN,NaN,7581,65958,40219.0,0.0,NaN,NaN
2,2,2021-01-14T17:00:00,3,Lombardia,3614,468,4082,52018,56100,1137,...,426098,26026,NaN,NaN,508224,5137863,2704477.0,26.0,NaN,NaN
3,19,2021-01-14T17:00:00,4,Trentino-Alto Adige,528,65,593,13090,13683,20,...,41056,1837,0.0,0.0,56576,863185,325308.0,1.0,0,NaN
4,3,2021-01-14T17:00:00,5,Veneto,2447,328,2775,77050,79825,-2145,...,206185,7785,NaN,NaN,293795,3526933,1274187.0,32.0,NaN,NaN
5,4,2021-01-14T17:00:00,6,Friuli Venezia Giulia,668,68,736,12127,12863,65,...,43723,2006,NaN,NaN,58592,1011978,361869.0,5.0,NaN,NaN
6,5,2021-01-14T17:00:00,7,Liguria,678,65,743,4331,5074,72,...,56812,3074,NaN,NaN,64960,758219,338106.0,5.0,I 191219 test antigenici rapidi sono riferiti ...,NaN
7,6,2021-01-14T17:00:00,8,Emilia-Romagna,2610,233,2843,53105,55948,-2032,...,132548,8590,NaN,NaN,197086,2752436,1285504.0,13.0,NaN,NaN
8,7,2021-01-14T17:00:00,9,Toscana,723,132,855,7830,8685,-141,...,113949,3930,NaN,NaN,126564,1998217,1103382.0,2.0,Positivi diagnosticati solo con test antigenic...,NaN
9,8,2021-01-14T17:00:00,10,Umbria,277,51,328,4161,4489,19,...,26510,678,NaN,NaN,31677,539805,264287.0,3.0,NaN,NaN


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione
0,2021-01-15T17:00:00,1,Piemonte,2522,163,2685,12121,14806,83,871,...,8319,NaN,NaN,211209,1785825,1025418.0,8.0,NaN,NaN,4341375
1,2021-01-15T17:00:00,2,Valle d'Aosta,40,3,43,373,416,4,20,...,392,NaN,NaN,7601,66212,40288.0,2.0,NaN,NaN,125501
2,2021-01-15T17:00:00,3,Lombardia,3601,466,4067,53087,57154,1054,2205,...,26094,NaN,NaN,510429,5171810,2713639.0,19.0,NaN,NaN,10103969
3,2021-01-15T17:00:00,4,Trentino-Alto Adige,513,65,578,13537,14115,432,891,...,1853,0.0,0.0,57467,873851,326301.0,6.0,0,NaN,1074819
4,2021-01-15T17:00:00,5,Veneto,2386,327,2713,72640,75353,-4472,1079,...,7859,NaN,NaN,294874,3584775,1278230.0,32.0,NaN,NaN,4907704
5,2021-01-15T17:00:00,6,Friuli Venezia Giulia,688,68,756,12123,12879,16,919,...,2035,NaN,NaN,59511,1022612,365548.0,6.0,NaN,NaN,1211357
6,2021-01-15T17:00:00,7,Liguria,668,59,727,4307,5034,-40,254,...,3084,NaN,NaN,65214,764447,340315.0,0.0,I 2243 test antigenici rapidi sono riferiti al...,NaN,1543127
7,2021-01-15T17:00:00,8,Emilia-Romagna,2511,231,2742,52768,55510,-438,1794,...,8657,NaN,NaN,198878,2774816,1289999.0,10.0,NaN,NaN,4467118
8,2021-01-15T17:00:00,9,Toscana,715,129,844,7754,8598,-87,446,...,3940,NaN,NaN,127010,2012434,1106215.0,7.0,NaN,NaN,3722729
9,2021-01-15T17:00:00,10,Umbria,278,50,328,4161,4489,0,232,...,682,NaN,NaN,31909,543318,265261.0,3.0,NaN,NaN,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione,totale_posti_terapia_intensiva
0,2021-01-15T17:00:00,1,Piemonte,2522,163,2685,12121,14806,83,871,...,NaN,NaN,211209,1785825,1025418.0,8.0,NaN,NaN,4341375,575
1,2021-01-15T17:00:00,2,Valle d'Aosta,40,3,43,373,416,4,20,...,NaN,NaN,7601,66212,40288.0,2.0,NaN,NaN,125501,20
2,2021-01-15T17:00:00,3,Lombardia,3601,466,4067,53087,57154,1054,2205,...,NaN,NaN,510429,5171810,2713639.0,19.0,NaN,NaN,10103969,1036
3,2021-01-15T17:00:00,4,Trentino-Alto Adige,513,65,578,13537,14115,432,891,...,0.0,0.0,57467,873851,326301.0,6.0,0,NaN,1074819,106
4,2021-01-15T17:00:00,5,Veneto,2386,327,2713,72640,75353,-4472,1079,...,NaN,NaN,294874,3584775,1278230.0,32.0,NaN,NaN,4907704,1016
5,2021-01-15T17:00:00,6,Friuli Venezia Giulia,688,68,756,12123,12879,16,919,...,NaN,NaN,59511,1022612,365548.0,6.0,NaN,NaN,1211357,180
6,2021-01-15T17:00:00,7,Liguria,668,59,727,4307,5034,-40,254,...,NaN,NaN,65214,764447,340315.0,0.0,I 2243 test antigenici rapidi sono riferiti al...,NaN,1543127,209
7,2021-01-15T17:00:00,8,Emilia-Romagna,2511,231,2742,52768,55510,-438,1794,...,NaN,NaN,198878,2774816,1289999.0,10.0,NaN,NaN,4467118,634
8,2021-01-15T17:00:00,9,Toscana,715,129,844,7754,8598,-87,446,...,NaN,NaN,127010,2012434,1106215.0,7.0,NaN,NaN,3722729,523
9,2021-01-15T17:00:00,10,Umbria,278,50,328,4161,4489,0,232,...,NaN,NaN,31909,543318,265261.0,3.0,NaN,NaN,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-01-15T17:00:00,3,Lombardia,3601,466,4067,53087,57154,1054,2205,...,44.98,1069,-382,33947,9162.0,68,1083,6.50,0.02182,5.05
1,2021-01-15T17:00:00,19,Sicilia,1403,210,1613,43432,45045,180,1945,...,35.71,169,78,24005,6316.0,39,1726,8.10,0.03915,2.36
2,2021-01-15T17:00:00,8,Emilia-Romagna,2511,231,2742,52768,55510,-438,1794,...,36.44,-337,279,22380,4495.0,67,2163,8.02,0.04016,4.45
3,2021-01-15T17:00:00,12,Lazio,2780,299,3079,74676,77755,-225,1394,...,35.30,-195,-422,28242,16200.0,36,1583,4.94,0.02377,3.19
4,2021-01-15T17:00:00,16,Puglia,1374,148,1522,54300,55822,287,1295,...,40.11,314,-229,9812,5614.0,31,977,13.20,0.03231,2.68
5,2021-01-15T17:00:00,15,Campania,1401,93,1494,71439,72933,180,1150,...,18.42,155,-144,13929,9004.0,37,933,8.26,0.01988,3.54
6,2021-01-15T17:00:00,5,Veneto,2386,327,2713,72640,75353,-4472,1079,...,32.19,-4410,-997,57842,4043.0,74,5477,1.87,0.02199,6.01
7,2021-01-15T17:00:00,6,Friuli Venezia Giulia,688,68,756,12123,12879,16,919,...,37.78,-4,114,10634,3679.0,29,874,8.64,0.07587,4.91
8,2021-01-15T17:00:00,4,Trentino-Alto Adige,513,65,578,13537,14115,432,891,...,61.32,447,445,10666,993.0,16,443,8.35,0.08290,5.35
9,2021-01-15T17:00:00,1,Piemonte,2522,163,2685,12121,14806,83,871,...,28.35,145,14,18046,4846.0,20,768,4.83,0.02006,4.87


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,65,41,106,61.32
1,Marche,82,61,143,57.34
2,Lombardia,466,570,1036,44.98
3,Umbria,50,67,117,42.74
4,Puglia,148,221,369,40.11
5,Friuli Venezia Giulia,68,112,180,37.78
6,Emilia-Romagna,231,403,634,36.44
7,Sicilia,210,378,588,35.71
8,Lazio,299,548,847,35.30
9,Veneto,327,689,1016,32.19


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,2021-01-13T17:00:00,22,P.A. Trento,320,44,364,1624,1988,15,175,21093,1037,NaN,NaN,24118,474659,155952.0,4.0,NaN,"Dei 175 casi odierni, 65 sono molecolari con p..."
650,2021-01-14T17:00:00,21,P.A. Bolzano,212,23,235,11370,11605,-70,174,19785,796,NaN,NaN,32186,386047,168946.0,1.0,NaN,NaN
651,2021-01-14T17:00:00,22,P.A. Trento,316,42,358,1720,2078,90,272,21271,1041,NaN,NaN,24390,477138,156362.0,0.0,NaN,"Dei 272 casi odierni, 191 sono molecolari con ..."
652,2021-01-15T17:00:00,21,P.A. Bolzano,209,23,232,11575,11807,202,529,20106,802,NaN,NaN,32715,393231,169639.0,2.0,NaN,NaN


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,2021-01-11T17:00:00,4,Trentino-Alto Adige,583,69,652,13052,13704,46,194,...,7.0,1108.0,510.0,28.0,10.58,4.76,17.51,38.04,-1.43,0.34
323,2021-01-12T17:00:00,4,Trentino-Alto Adige,565,68,633,13036,13669,-35,333,...,14.0,3089.0,891.0,-16.0,10.74,4.63,10.78,37.37,-1.45,-0.26
324,2021-01-13T17:00:00,4,Trentino-Alto Adige,532,67,599,13064,13663,-6,444,...,13.0,5363.0,935.0,28.0,11.19,4.38,8.28,47.49,-1.47,-0.04
325,2021-01-14T17:00:00,4,Trentino-Alto Adige,528,65,593,13090,13683,20,446,...,12.0,4667.0,899.0,26.0,10.96,4.33,9.56,49.61,-2.99,0.15


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
